<a href="https://colab.research.google.com/github/xxp-nlp/add_tag_algorithm/blob/main/2021_8_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [498]:
import re

filename = '/content/CiWoTestdata-Correct.txt'

dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
             (-?\d+)      # 数字(係り先)
                      ''', re.VERBOSE)

class Morph:
    def __init__(self, line):
        cols = line.split(' ')
        self.surface = cols[0] # 表層形(surface)

class Chunk_correct:
    def __init__(self, morphs, dst, id):
        self.morphs = morphs
        self.dst  = dst  # 係り先文節インデックス番号
        self.id = id
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節
        self.trim_phrase = self.phrase.replace('、','')
        self.tag = 'KEEP'

sentences = []
chunks = []
morphs = []
ids = []
dic_correct = {}
i = 0
with open(filename, mode='r') as f:
  for line in f:  # 1行ずつ読込
    dependancies = dependancy.match(line)
    fields = line.split(' ')


    if fields[0] == '#':
      id = fields[1]
      continue

    # EOSまたは係り受け解析結果でない場合
    if not (line == 'EOS\n' or dependancies):
      morphs.append(Morph(line))


    # EOSまたは係り受け解析結果で、形態素解析結果がある場合
    elif len(morphs) > 0:
      chunks.append(Chunk_correct(morphs, dst, id))
      morphs = []  

    # 係り受け結果の場合
    if dependancies:
      dst = int(dependancies.group(1))

    # EOSで係り受け結果がある場合
    if line == 'EOS\n' and len(chunks) > 0:
      sentences.append(chunks)
      chunks = []
      dic_correct[i] = [sentences,id]
      i+=1

for i, chunk in enumerate(sentences[9]):
    print('{}: {}  係り先:{} {}'.format(i, chunk.trim_phrase, chunk.dst, chunk.tag))

0: 同大統領の  係り先:1 KEEP
1: 首都脱出は  係り先:9 KEEP
2: 先に  係り先:9 KEEP
3: インタファクス通信が  係り先:9 KEEP
4: 六日  係り先:9 KEEP
5: グロズヌイの  係り先:6 KEEP
6: 軍事筋の  係り先:7 KEEP
7: 情報と  係り先:8 KEEP
8: して  係り先:9 KEEP
9: 伝えたが  係り先:13 KEEP
10: ロシア政府が  係り先:12 KEEP
11: 公式に  係り先:12 KEEP
12: 認めたのは  係り先:13 KEEP
13: 初めて。  係り先:-1 KEEP


In [499]:
filename = '/content/CiWoTestdata.txt'

dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
             (-?\d+)      # 数字(係り先)
                      ''', re.VERBOSE)

class Morph:
    def __init__(self, line):
        cols = line.split(' ')
        self.surface = cols[0] # 表層形(surface)


class Chunk_wrong:
    def __init__(self, morphs, dst, id):
        self.morphs = morphs
        self.dst  = dst  # 係り先文節インデックス番号
        self.id = id
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節
        self.trim_phrase = self.phrase.replace('、','')
        self.tag = 'KEEP'

dic_wrong = {}
sentences_wrong = []
chunks = []
morphs = []
ids = []
i = 0
with open(filename, mode='r') as f:
  for line in f:  # 1行ずつ読込
    dependancies = dependancy.match(line)
    fields = line.split(' ')


    if fields[0] == '#':
      id = fields[1]
      continue

    # EOSまたは係り受け解析結果でない場合
    if not (line == 'EOS\n' or dependancies):
      morphs.append(Morph(line))


    # EOSまたは係り受け解析結果で、形態素解析結果がある場合
    elif len(morphs) > 0:
      chunks.append(Chunk_wrong(morphs, dst, id))
      morphs = []  

    # 係り受け結果の場合
    if dependancies:
      dst = int(dependancies.group(1))

    # EOSで係り受け結果がある場合
    if line == 'EOS\n' and len(chunks) > 0:
      sentences_wrong.append(chunks)
      chunks = []
      dic_wrong[i] = [sentences_wrong,id]
      i+=1


for i, chunk in enumerate(sentences_wrong[1]):
    print('{}: {}  係り先:{} {}'.format(i, chunk.trim_phrase, chunk.dst, chunk.tag))

0: 党全体での  係り先:1 KEEP
1: 新党移行を  係り先:4 KEEP
2: 今春の  係り先:3 KEEP
3: 統一地方選後に  係り先:4 KEEP
4: 目指す  係り先:5 KEEP
5: 考えを  係り先:6 KEEP
6: 強調  係り先:12 KEEP
7: 離党など  係り先:9 KEEP
8: 性急な  係り先:9 KEEP
9: 行動への  係り先:10 KEEP
10: 自重を  係り先:12 KEEP
11: 首相は  係り先:12 KEEP
12: 求めた。  係り先:-1 KEEP


In [500]:
# scale the data
lenth = min(len(sentences),len(sentences_wrong))

print("読みやすい文：",len(sentences))
print("読みにくい文：",len(sentences_wrong))

correct = []
wrong = []

for i in range(len(sentences)):
  for j in range(len(sentences_wrong)):
    if dic_correct[i][1] == dic_wrong[j][1]:  
      correct.append(sentences[i])
      wrong.append(sentences_wrong[j])
      


読みやすい文： 530
読みにくい文： 546


In [501]:
def algo(wrong,correct):
  dic2 = {}

  for i in range(2):

    for j in range(len(wrong[i])):
      if wrong[i][j].trim_phrase not in dic2:
        dic2[wrong[i][j].trim_phrase] = [] 
      dic2[wrong[i][j].trim_phrase].append(j)


    for j in range(len(correct[i])):
      if j == len(correct[i])-1:
        break

      for m in range(len(dic2[correct[i][j].trim_phrase])):
        for n in range(len(dic2[correct[i][j+1].trim_phrase])):
          if correct[i][j].dst == j+1 and dic2[correct[i][j].trim_phrase][m] == dic2[correct[i][j+1].trim_phrase][n] - 1:
            wrong[i][dic2[correct[i][j].trim_phrase][m]].tag = "REDUCE"
            correct[i][j].tag = "REDUCE"

          if dic2[correct[i][j].trim_phrase][m] < len(wrong[i])-1 and dic2[correct[i][j].trim_phrase][m] == dic2[correct[i][j+1].trim_phrase][n] + 1: 
            wrong[i][dic2[sentences[i][j].trim_phrase][m]].tag = "SWAP_B"
            wrong[i][dic2[sentences[i][j+1].trim_phrase][n]].tag = "SWAP_F"

    dic2 = {}

In [502]:
algo(wrong,correct)

In [503]:
# step1の結果確認
for i in range(9,10):
  print("-------  sentence ", i+1, "-------")
  for j, chunk in enumerate(wrong[i]):
    print('{}: {}: {}'.format(j, wrong[i][j].trim_phrase, wrong[i][j].tag))

-------  sentence  10 -------
0: 公式に: KEEP
1: ロシア政府が: KEEP
2: 認めたのは: KEEP
3: 同大統領の: KEEP
4: 首都脱出は: KEEP
5: 先に: KEEP
6: 六日: KEEP
7: グロズヌイの: KEEP
8: 軍事筋の: KEEP
9: 情報と: KEEP
10: して: KEEP
11: インタファクス通信が: KEEP
12: 伝えたが: KEEP
13: 初めて。: KEEP


In [504]:
# step1の結果確認
for i in range(9,10):
  print("-------  sentence ", i+1, "-------")
  for j, chunk in enumerate(correct[i]):
    print('{}: {}: {}'.format(j, chunk.trim_phrase, chunk.tag))

-------  sentence  10 -------
0: 同大統領の: KEEP
1: 首都脱出は: KEEP
2: 先に: KEEP
3: インタファクス通信が: KEEP
4: 六日: KEEP
5: グロズヌイの: KEEP
6: 軍事筋の: KEEP
7: 情報と: KEEP
8: して: KEEP
9: 伝えたが: KEEP
10: ロシア政府が: KEEP
11: 公式に: KEEP
12: 認めたのは: KEEP
13: 初めて。: KEEP


In [505]:
class Chunk_temp:
    def __init__(self, trim_phrase, tag ,dst):
      self.trim_phrase = trim_phrase
      self.tag = tag
      self.dst = dst

class New_chunk:
    def __init__(self, new_chunk):
      self.trim_phrase = new_chunk[0]
      self.tag = new_chunk[1]
      self.dst = new_chunk[2]

    def change(self):
      chunks = []
      
      for i in range(len(self.dst)):
#        print(self.trim_phrase[i], self.tag[i], self.dst[i])
        chunks.append(Chunk_temp(self.trim_phrase[i], self.tag[i], self.dst[i]))
#        print("------------")
      return chunks



class Merged:
  def __init__(self, j, phrase, trim_phrase, tag, dst):
    self.j = j
    self.phrase = phrase
    self.trim_phrases = trim_phrase
    self.tag = tag
    self.dst = dst

  def merge(self):
    new_phrase = []
    new_tag = []
    index = []
    set1 = set([])
    str1 = ''
    list_dst = []

    for i in range(j+1):
      if tag[i] == 'REDUCE':
        set1.add(i)
        set1.add(i+1)

      if tag[i] == 'KEEP':
        temp_list = list(set1)
        temp_list.sort()
        if len(temp_list)>0:
#          print(dst[temp_list[-1]])
          list_dst.append(dst[temp_list[-1]])
        for i in temp_list:
          str1 += trim_phrase[i]
#          print(i,str1)
        if len(str1)>0:
          new_phrase.append(str1)
          index.append(list(set1))
          new_tag.append("KEEP")
          str1 = ''
          set1.clear()
        else: 
          new_phrase.append(trim_phrase[i])
          new_tag.append("KEEP")
          index.append(i)
#          print(dst[i])
          list_dst.append(dst[i])
          

      if tag[i] == 'SWAP_F':
        new_phrase.append(trim_phrase[i+1])
        new_tag.append("KEEP")
        index.append(i+1)
      if tag[i] == 'SWAP_B':
        new_phrase.append(trim_phrase[i-1])
        new_tag.append("KEEP")
        index.append(i-1)

    new_dst = []
    for m in range(len(list_dst)):
      for n in range(len(index)):
        if isinstance(index[n],int):
          if list_dst[m] == index[n]:
#            print(n)
            new_dst.append(n)

        else:
          for k in range(len(index[n])):
            if list_dst[m] == index[n][k]:
#              print(n)
              new_dst.append(n)

    new_dst.append(-1)

    return new_phrase,new_tag,new_dst



    
# main

phrase = []
trim_phrase = []
tag = []
dst = []

new_chunks = []

correct_new_chunks = []


for i in range(2):
   
  for j, chunk in enumerate(wrong[i]):
    phrase.append(chunk.phrase)
    trim_phrase.append(chunk.trim_phrase)
    tag.append(chunk.tag)
    dst.append(chunk.dst)

  new_chunk = Merged(j, phrase, trim_phrase, tag, dst).merge()
  phrase = []
  trim_phrase = []
  tag = []
  dst = []
  temp_chunk = New_chunk(new_chunk).change()
  new_chunks.append(temp_chunk)

  for j, chunk in enumerate(correct[i]):
    phrase.append(chunk.phrase)
    trim_phrase.append(chunk.trim_phrase)
    tag.append(chunk.tag)
    dst.append(chunk.dst)
  
  # print(trim_phrase)  
  # print(tag)
  # print(dst)    

  correct_new_chunk = Merged(j, phrase, trim_phrase, tag, dst).merge()
  phrase = []
  trim_phrase = []
  tag = []
  dst = []
  temp_correct_chunk = New_chunk(correct_new_chunk).change()
  correct_new_chunks.append(temp_correct_chunk)


In [506]:
for i in range(2):
  print('------------sentence:',i+1,'----------------')
  for j in range(len(new_chunks[i])):
    print(j, new_chunks[i][j].trim_phrase, new_chunks[i][j].tag, new_chunks[i][j].dst)

------------sentence: 1 ----------------
0 同社では KEEP 4
1 準役員クラス以上に KEEP 4
2 能力重視型の年俸制を KEEP 4
3 すでに KEEP 4
4 導入している。 KEEP -1
------------sentence: 2 ----------------
0 党全体での新党移行を KEEP 2
1 今春の統一地方選後に KEEP 2
2 目指す考えを強調 KEEP 6
3 離党など KEEP 4
4 性急な行動への自重を KEEP 6
5 首相は KEEP 6
6 求めた。 KEEP -1


In [507]:
# new_chunks[0][2].trim_phrase = 'すでに'
# new_chunks[0][3].trim_phrase = '能力重視型の年俸制を'

In [508]:
# for i in range(1):
#   print('------------sentence:',i+1,'----------------')
#   for j in range(len(new_chunks[i])):
#     print(j, new_chunks[i][j].trim_phrase, new_chunks[i][j].tag, new_chunks[i][j].dst)

In [509]:
for i in range(2):
  print('------------sentence:',i+1,'----------------')
  for j in range(len(correct_new_chunks[i])):
    print(j, correct_new_chunks[i][j].trim_phrase, correct_new_chunks[i][j].tag, correct_new_chunks[i][j].dst)

------------sentence: 1 ----------------
0 同社では KEEP 4
1 すでに KEEP 4
2 準役員クラス以上に KEEP 4
3 能力重視型の年俸制を KEEP 4
4 導入している。 KEEP -1
------------sentence: 2 ----------------
0 首相は KEEP 6
1 今春の統一地方選後に KEEP 3
2 党全体での新党移行を KEEP 3
3 目指す考えを強調 KEEP 6
4 離党など KEEP 5
5 性急な行動への自重を KEEP 6
6 求めた。 KEEP -1


In [510]:
algo(new_chunks,correct_new_chunks)

KeyError: ignored

In [ ]:
for i in range(2):
  print('------------sentence:',i+1,'----------------')
  for j in range(len(new_chunks[i])):
    print(j, new_chunks[i][j].trim_phrase, new_chunks[i][j].tag, new_chunks[i][j].dst)